## Import the need libraries

In [1]:
import requests
import json
import pandas as pd


### After exploring the website https://liveresults.civichive.org/ using the web developers tool, it was discovered that the website uses hidden api to get the data it displays. 

In [2]:
states_url='https://liveresultsapi.civichive.org/api/v1/states' #url to the hidden APi
states_raw=requests.get(states_url).text #make request to the api and get response as text
states_data=json.loads(states_raw) #convert the response to json
states_slug=[i['slug'] for i in states_data['data']] #get the slug name for each state(to access each states data)
states_name=[i['name'] for i in states_data['data']] #get the name for each state

nation_df=[]#create a list to save each states data



    
    


### This loop goes over each state and retrieve the state's election data for the presidential election

In [3]:
# Loop through each state name and its corresponding slug
for state, slug in zip(states_name, states_slug):

    # Build the URL to retrieve election results for the current state and year
    result_url = f'https://liveresultsapi.civichive.org/api/v1/election/{slug}/2023/data'

    # Make a request to the API and parse the response as JSON
    result_raw = requests.get(result_url).text
    result_data = json.loads(result_raw)

    # Extract the number of votes for each political party from the response data
    party_votes = {d['abbr']: d['vote'] for d in result_data['data']['presidential']['parties']}

    # Extract the total number of registered voters and votes cast in the election
    reg_voters = result_data['data']['presidential']['registeredVote']
    total_votes = result_data['data']['presidential']['totalVote']

    # Create a DataFrame for the current state with the election results
    state_df = pd.DataFrame([[state] + list(party_votes.values()) + [total_votes] + [reg_voters]], 
                            columns=['State'] + list(party_votes.keys()) + ['Total Votes', 'Registered Voters'])

    # Append the state DataFrame to the national DataFrame
    nation_df.append(state_df)


### Concatenate all states dataframe in list nation_df into oe, format it and save it as election data.csv

In [5]:
final=pd.concat(nation_df)
final=final.reset_index().drop('index', axis=1)#reset the index
final.to_csv('election data.csv', index=False)#save it